Importar librerias

In [4]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import emoji
import flet
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import pickle
import re


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Transformar emojis a texto

In [7]:
with pd.read_csv('data\Datathon 2024 - Reto Hey - Dataset Puìblico - Sheet1.csv') as file:
    reader = csv.reader(file)

    for row in reader:
        print(emoji.demojize(str(row), language='es'))

['date', 'time', 'tweet']
['01/01/2023', '', 'Resuelto, muchísimas gracias , excelente servicio como siempre']
['02/01/2023', '', 'Muchas gracias, espero su dm']
['02/01/2023', '', 'Muchas gracias!']
['02/01/2023', '', 'Algo similar me paso. Quería renovar mi token para poder hacer una transferencia antes de cerrar la cuenta y wow. Fue más fácil sacar el dinero por ventanilla.']
['02/01/2023', '', 'Yeeeei! a través de mi cuenta en acabo de contratar mi cuenta de ahorro con mi reto de 2023 para tener 25 mil pesos al final del año. Tiene una súper herramienta para domiciliar a mi cuenta eje y así ahorrar disciplinadamente cada semana! Vamos !!! :bíceps_flexionado_tono_de_piel_claro_medio::bolsa_de_dinero:']
['02/01/2023', '', 'Esta es la señal que necesitan para tener el mínimo dinero en banamex']
['03/01/2023', '', 'Gracias :cara_radiante_con_ojos_sonrientes:']
['03/01/2023', '', 'Muchas gracias :cara_feliz_con_ojos_sonrientes:']
['03/01/2023', '', 'Yo no sé mucho, pero te recomiendo la

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HP\AppData\Local\Temp\ipykernel_3284\1145245214.py:1: SyntaxWarning: invalid escape sequence '\D'
  with open('data\Datathon 2024 - Reto Hey - Dataset Puìblico - Sheet1.csv', 'r', encoding='utf8') as file:
